In [1]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import make_pipeline

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../datasets/BankChurners_change.csv', encoding='CP949')
data = data.iloc[:,2:23]
np.random.seed(42)
random.seed(42)

In [3]:
object_columns = data.select_dtypes('object').columns

for i in object_columns:

    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']

category : [0 1]
classes : ['F' 'M']

category : [0 1 2 3 4 5 6]
classes : ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated' 'Unknown']

category : [0 1 2 3]
classes : ['Divorced' 'Married' 'Single' 'Unknown']

category : [0 1 2 3 4 5]
classes : ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K'
 'Unknown']

category : [0 1 2 3]
classes : ['Blue' 'Gold' 'Platinum' 'Silver']

category : [0 1 2 3 4]
classes : ['20대' '30대' '40대' '50대' '60대 이상']



## Under Sampling

In [4]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i == OneSidedSelection:
        sampling = i(random_state=42)
    else:
        sampling = i()

    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    rf = RandomForestClassifier(random_state=42)
    rf.fit(x_train_ss, y_train)
    pred = rf.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("RandomForest Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

RandomForest Model's score by Under sampling


,f1_score,recall,precision,accuracy
NeighbourhoodCleaningRule,0.976581,0.981754,0.971462,0.960513
OneSidedSelection,0.976163,0.988228,0.964388,0.959526
EditedNearestNeighbours,0.975581,0.975868,0.975294,0.959033
TomekLinks,0.974985,0.986463,0.963772,0.957552
AllKNN,0.973739,0.971160,0.976331,0.956071
RepeatedEditedNearestNeighbours,0.971598,0.966451,0.976800,0.952616
NearMiss,0.888745,0.806357,0.989884,0.830701


## Over Sampling

In [5]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy, random_state=42)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        rf = RandomForestClassifier(random_state=42)
        rf.fit(x_train_ss, y_train)
        pred = rf.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("RandomForest Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

RandomForest Model's score by Over sampling


,f1_score,recall,precision,accuracy
BorderlineSMOTE(0.3),0.979185,0.982931,0.975467,0.964956
SMOTE(0.3),0.977479,0.983520,0.971512,0.961994
BorderlineSMOTE(0.5),0.977226,0.972337,0.982164,0.961994
BorderlineSMOTE(0.4),0.976978,0.974102,0.979870,0.961500
SMOTE(0.6),0.976964,0.973514,0.980439,0.961500
SMOTE(0.4),0.976717,0.975280,0.978158,0.961007
ADASYN(0.5),0.976275,0.968805,0.983861,0.960513
ADASYN(0.3),0.976009,0.981754,0.970332,0.959526
ADASYN(0.4),0.975797,0.972925,0.978686,0.959526
SMOTE(0.5),0.975754,0.971160,0.980392,0.959526


## Combine Sampling

In [6]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i(random_state=42)
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    rf = RandomForestClassifier(random_state=42)
    rf.fit(x_train_ss, y_train)
    pred = rf.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[sampling] = [f1, recall, precision, acc]

print("RandomForest Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

RandomForest Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTETomek(random_state=42),0.970527,0.959388,0.981928,0.951135
SMOTEENN(random_state=42),0.950900,0.917599,0.986709,0.920533


## Pipeline을 통한 Combine Sampling

In [7]:
strategy_ = [0.3, 0.4, 0.5]
columns = ['f1_score', 'recall', 'precision', 'accuracy']

df = pd.DataFrame(columns=columns)

under_sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]
over_sampling = [SMOTE, BorderlineSMOTE, ADASYN]

rf = RandomForestClassifier()

for over in over_sampling:

    for under in under_sampling:

        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        for strategy in strategy_:
            
            if under == OneSidedSelection:
                under_ = under(random_state=42)
            else:
                under_ = under()

            over_ = over(sampling_strategy=strategy, random_state=42)

            ss = StandardScaler()
            x_train_ss = ss.fit_transform(x_train)
            x_test_ss = ss.fit_transform(x_test)

            model = make_pipeline(over_, under_, rf)   
            model.fit(x_train_ss, y_train)
            pred = model.predict(x_test_ss)

            acc = accuracy_score(y_test, pred)
            recall = recall_score(y_test, pred)
            precision = precision_score(y_test, pred)
            f1 = f1_score(y_test, pred)

            df.loc[over.__name__+under.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("RandomForest Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

RandomForest Model's score by sampling


,f1_score,recall,precision,accuracy
BorderlineSMOTETomekLinks(0.3),0.979759,0.982931,0.976608,0.965943
BorderlineSMOTEOneSidedSelection(0.3),0.978561,0.980577,0.976553,0.963968
BorderlineSMOTETomekLinks(0.4),0.978472,0.976457,0.980496,0.963968
ADASYNOneSidedSelection(0.3),0.978274,0.980577,0.975981,0.963475
BorderlineSMOTEOneSidedSelection(0.4),0.978132,0.974102,0.982196,0.963475
BorderlineSMOTETomekLinks(0.5),0.977172,0.969982,0.984468,0.961994
SMOTEOneSidedSelection(0.4),0.976991,0.974691,0.979302,0.961500
SMOTETomekLinks(0.3),0.976894,0.982931,0.970930,0.961007
ADASYNTomekLinks(0.3),0.976812,0.979400,0.974239,0.961007
SMOTETomekLinks(0.4),0.976429,0.975280,0.977581,0.960513


## Under, Over, Combine Sampling과 Pipeline을 통한 Combine Sampling 전체 결과

In [8]:
sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i in [OneSidedSelection, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]:
        sampling = i(random_state=42)
    else:
        sampling = i()
        
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    rf = RandomForestClassifier(random_state=42)
    rf.fit(x_train_ss, y_train)
    pred = rf.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[i.__name__] = [f1, recall, precision, acc]

print("RandomForest Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

RandomForest Model's score by sampling


,f1_score,recall,precision,accuracy
BorderlineSMOTETomekLinks(0.3),0.979759,0.982931,0.976608,0.965943
BorderlineSMOTEOneSidedSelection(0.3),0.978561,0.980577,0.976553,0.963968
BorderlineSMOTETomekLinks(0.4),0.978472,0.976457,0.980496,0.963968
ADASYNOneSidedSelection(0.3),0.978274,0.980577,0.975981,0.963475
BorderlineSMOTEOneSidedSelection(0.4),0.978132,0.974102,0.982196,0.963475
BorderlineSMOTETomekLinks(0.5),0.977172,0.969982,0.984468,0.961994
SMOTEOneSidedSelection(0.4),0.976991,0.974691,0.979302,0.961500
SMOTETomekLinks(0.3),0.976894,0.982931,0.970930,0.961007
ADASYNTomekLinks(0.3),0.976812,0.979400,0.974239,0.961007
NeighbourhoodCleaningRule,0.976581,0.981754,0.971462,0.960513


## No Sampling

In [9]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
NO_Sampling = pd.DataFrame(columns=columns)

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.fit_transform(x_test)


rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
pred = rf.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

NO_Sampling.loc['No Sampling'] = [f1, recall, precision, acc]

print("RandomForest Model's score by sampling")
NO_Sampling.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
NO_Sampling.head(10)

RandomForest Model's score by sampling


,f1_score,recall,precision,accuracy
No Sampling,0.976149,0.98764,0.964922,0.959526
